In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import random
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
# Declaring constants
TRAIN_VAL_FACTOR = 0.2

In [ ]:
def load_img_src(folder):
    arr_img_src = []

    for item in os.listdir(folder):
        if item.endswith(".jpg"):
            arr_img_src.append(f"{folder}/{item}")

    return arr_img_src

In [ ]:
def preprocess(dict_img_src):
    pile = []

    # We create a big pile of dicts with x and y
    for key in ["Healthy", "Sick"]:
        for img_src in dict_img_src[key]:
            print(f"Preprocessing {img_src}")

            if key == "Healthy":
                label = int(1)
            else:
                label = int(0)
            
            arr_img = cv2.imread(img_src, cv2.IMREAD_REDUCED_GRAYSCALE_8)
            arr_img = cv2.resize(arr_img, (255, 255)) / 255 # /255 to normalize this image data
            pile.append(np.array([arr_img, label]))

    # We shuffle the pile to create a random order of healthy and sick cells
    random.shuffle(pile)

    # We create and empty np array for x (data) and y (label)
    # x_pile = np.empty((len(pile), 255, 255))
    x_pile = np.empty((0, 255, 255))
    y_pile = np.array([])

    # The pile is an array of dicts
    # We need to unpack this and place in the 3d np array for x and y
    print("Formatting data to tensor matrix...")
    for i in pile:
        x_pile = np.append(x_pile, [i[0]], axis=0)
        y_pile = np.append(y_pile, i[1])
  
    # Now that we seperated the x and y, we create the training and validation sets
    print("Creating train and test batches...")

    # # Training validation data and labels
    x_train = x_pile[:int((1 - TRAIN_VAL_FACTOR) * len(x_pile))] # Images
    y_train = y_pile[:int((1 - TRAIN_VAL_FACTOR) * len(y_pile))] # Labels

    # Validation data and labels
    x_val = x_pile[int((1 - TRAIN_VAL_FACTOR) * len(y_pile)):] # Images
    y_val = y_pile[int((1 - TRAIN_VAL_FACTOR) * len(y_pile)):] # Labels

    print("Done")

    return ((x_train, x_val), (y_train, y_val))
    

In [ ]:
# Prepocessing all data into usable chunks for CNN
dict_img_src = { "Healthy": load_img_src("healthy"), "Sick": load_img_src("sick") }
(x_train, x_val), (y_train, y_val) = preprocess(dict_img_src)

x_train = x_train.reshape((-1, 255, 255, 1)) # Reshaping to (length_data_set, width_img, height_img, channels)
x_val = x_val.reshape((-1, 255, 255, 1))

np.save("x_train", x_train)
np.save("x_val", x_val)
np.save("y_train", y_train)
np.save("y_val", y_val)


print(f"x_train: {x_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"x_val: {x_val.shape}")
print(f"y_val: {y_val.shape}")

In [ ]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")
x_val = np.load("x_val.npy")
y_val = np.load("y_val.npy")

In [ ]:
# Defining CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), padding="same", activation="relu", input_shape=(255, 255, 1)))
model.add(MaxPooling2D(pool_size=(5, 5)))
model.add(Flatten())
model.add(Dense(1500, activation="relu"))
model.add(Dense(750, activation="relu"))
model.add(Dense(350, activation="relu"))
model.add(Dense(1, activation="sigmoid")) # 1 for firing or not firing, sigmoid because
                                          # binary classification

In [ ]:
# Compiling the model using binary_crossentropy and the adam optimizer
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Training the model using training data x and saving the metrics to the arr_metrics variable
arr_metrics = model.fit(x_train, y_train, batch_size=25, epochs=6)

In [ ]:
# Now we evaluate the model using the validation data y
model.evaluate(x_val, y_val)

In [ ]:
# Saving the model in project root directory
model.save("rbc_model.h5")

In [ ]:
print(arr_metrics.history.keys())

# Showing metrics saved in arr_metrics
plt.plot(arr_metrics.history["accuracy"])
plt.plot(arr_metrics.history["val_accuracy"])
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train", "Val"])
plt.show()

plt.plot(arr_metrics.history["loss"])
plt.plot(arr_metrics.history["val_loss"])
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend("Traing", "Val")
plt.show()